# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [ ]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')


# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.


Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He was driving for Red Bull Racing at the time.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [12]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [13]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [15]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [16]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [22]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [30]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In [33]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1. In your answers, you give context about what was happening in the F1 world that time around. You also give context about the pilot you mention, like other prices won or important facts of their history. Tell some fun facts about the competition that happened same year.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

The 2010 Formula 1 season was won by Sebastian Vettel, driving for Red Bull Racing. It was the German driver's first World Drivers' Championship title. Vettel had an impressive season, winning a total of 5 races that year.

The 2010 season was characterized by intense competition, with five drivers still in contention for the championship title as the season finale approached in Abu Dhabi. Vettel secured the championship by winning the final race of the season.

One of the notable moments of the 2010 season was the intense rivalry between Vettel and Fernando Alonso of Ferrari. The two drivers were the main contenders for the championship, and their battle on the track added excitement to the season.

Overall, the 2010 season was a thrilling and closely contested championship, with multiple drivers and teams fighting for victory throughout the year. It was a memorable season that showcased the competitive spirit of Formula 1.


In [37]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are a gardening expert.

     How often should I water my ficus?
     During the cold season: once every two weeks.
     During the warm season: every week."""}
]
print(return_OAIResponse("How often should I water my cactus?", context_user))

For most cactus plants, you should water them sparingly. Generally, during the growing season (spring and summer), you can water them once every two to three weeks. In the winter months when the cactus is dormant, you can reduce watering to every four to six weeks. The key is to allow the soil to dry out completely between waterings to prevent root rot.


In [41]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are a gardening expert.

     How often should I water my ficus?
     During the cold season: once every two weeks.
     During the warm season: every week.
     
     How often does my ficus need water?
     During colder periods: only once every two weeks
     During warmer periods: once a week"""}
]
print(return_OAIResponse("How often should I water my cactus?", context_user))

Cacti are known for their ability to store water in their stems and leaves, making them quite resilient to drought. Here are some general guidelines for watering cacti:

1. **Frequency**: During the growing season (spring and summer), water your cactus about once every two to four weeks. In the dormant season (fall and winter), reduce watering to about once a month or even less, depending on the specific conditions.

2. **Observation**: It's essential to observe your cactus and the soil it's planted in. Before watering, make sure the soil has completely dried out. Inserting a finger an inch or two into the soil can help you determine if it's time to water.

3. **Watering Technique**: When you do water, give the cactus a thorough soaking, allowing excess water to drain out of the container's bottom. Avoid leaving the cactus sitting in a saucer of water as this can lead to root rot.

4. **Seasonal Adjustments**: Adjust your watering schedule based on environmental factors such as tempera

In [44]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in gardening.\n\n'},
    {'role':'user', 'content':'How often should I water my ficus?'},
    {'role':'assistant', 'content':"""During the cold season: once every two weeks. \nTDuring the warm season: every week. """},
    {'role':'user', 'content':'How often does my ficus need water?'},
    {'role':'assistant', 'content':"""During colder periods: only once every two weeks. \nDuring warmer periods: once a week """},
]

print(return_OAIResponse("How often should I water my cactus?", context_user))

Cacti have specific watering needs due to their succulent nature. During the growing season in spring and summer, you should water your cactus more frequently, about once a week or when the soil is completely dry. In the fall and winter, you can reduce watering to once every 2-3 weeks or even less, as cacti enter a period of dormancy. Be sure to adjust your watering schedule based on the individual needs of your cactus and the environment it is growing in.
